[MeloTTS audio dataset maker](https://github.com/myshell-ai/MeloTTS/blob/main/docs/training.md)

In [ ]:
#@title Install and Restart
%cd /content/
!rm -rf /content/whisperX
!git clone https://github.com/m-bain/whisperX.git
%cd /content/whisperX
!pip install .
!echo /usr/lib64-nvidia/ >/etc/ld.so.conf.d/libcuda.conf; ldconfig
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64:/usr/local/cuda/lib
!pip install srt
!pip install deep_translator==1.11.4
!pip install edge_tts==6.1.7
!pip install gradio==4.19.2
!pip install pydub==0.25.1
!pip install pysrt==1.1.2
!pip install librosa==0.10.2.post1
!pip install yt-dlp==2024.2.16.232705.dev0
!pip install ffmpeg==1.4
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install "audio-separator[gpu]"
from IPython.display import clear_output
clear_output()
import time
time.sleep(5)
import os
os.kill(os.getpid(), 9)

In [1]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }

%%html
<b>Press play on the music player to keep the tab alive, then run the cell below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

# Multispeaker Identification using  pyannote-audio
Visit these links and accepet then generate Huggingface Access Token<br>
https://huggingface.co/pyannote/segmentation-3.0 <br>
https://huggingface.co/pyannote/speaker-diarization-3.1

In [2]:
huggingface_token = 'hf_bjkdasbnfkjdbnajkdsfnbjkfdsbkjdfs'  # @param {type: "string"}



In [3]:
#@title whisper config
import whisperx
import gc
import torch
import copy
from whisperx.alignment import (
    DEFAULT_ALIGN_MODELS_TORCH as DAMT,
    DEFAULT_ALIGN_MODELS_HF as DAMHF,
)
languages = {
    "None": "None",
    "Afrikaans": "af",
    "Amharic": "am",
    "Arabic": "ar",
    "Azerbaijani": "az",
    "Bulgarian": "bg",
    "Bengali": "bn",
    "Bosnian": "bs",
    "Catalan": "ca",
    "Czech": "cs",
    "Welsh": "cy",
    "Danish": "da",
    "German": "de",
    "Greek": "el",
    "English": "en",
    "Spanish": "es",
    "French": "fr",
    "Irish": "ga",
    "Galician": "gl",
    "Gujarati": "gu",
    "Hebrew": "he",
    "Hindi": "hi",
    "Croatian": "hr",
    "Hungarian": "hu",
    "Indonesian": "id",
    "Icelandic": "is",
    "Italian": "it",
    "Japanese": "ja",
    "Javanese": "jv",
    "Georgian": "ka",
    "Kazakh": "kk",
    "Khmer": "km",
    "Kannada": "kn",
    "Korean": "ko",
    "Lao": "lo",
    "Lithuanian": "lt",
    "Latvian": "lv",
    "Macedonian": "mk",
    "Malayalam": "ml",
    "Mongolian": "mn",
    "Marathi": "mr",
    "Malay": "ms",
    "Maltese": "mt",
    "Burmese": "my",
    "Norwegian Bokmål": "nb",
    "Nepali": "ne",
    "Dutch": "nl",
    "Polish": "pl",
    "Pashto": "ps",
    "Portuguese": "pt",
    "Romanian": "ro",
    "Russian": "ru",
    "Sinhala": "si",
    "Slovak": "sk",
    "Slovenian": "sl",
    "Somali": "so",
    "Albanian": "sq",
    "Serbian": "sr",
    "Sundanese": "su",
    "Swedish": "sv",
    "Swahili": "sw",
    "Tamil": "ta",
    "Telugu": "te",
    "Thai": "th",
    "Turkish": "tr",
    "Ukrainian": "uk",
    "Urdu": "ur",
    "Uzbek": "uz",
    "Vietnamese": "vi",
    "Chinese": "zh",
    "Zulu": "zu"
}

LANGUAGES_UNIDIRECTIONAL = {
    "Aymara (ay)": "ay",
    "Bambara (bm)": "bm",
    "Cebuano (ceb)": "ceb",
    "Chichewa (ny)": "ny",
    "Divehi (dv)": "dv",
    "Dogri (doi)": "doi",
    "Ewe (ee)": "ee",
    "Guarani (gn)": "gn",
    "Iloko (ilo)": "ilo",
    "Kinyarwanda (rw)": "rw",
    "Krio (kri)": "kri",
    "Kurdish (ku)": "ku",
    "Kirghiz (ky)": "ky",
    "Ganda (lg)": "lg",
    "Maithili (mai)": "mai",
    "Oriya (or)": "or",
    "Oromo (om)": "om",
    "Quechua (qu)": "qu",
    "Samoan (sm)": "sm",
    "Tigrinya (ti)": "ti",
    "Tsonga (ts)": "ts",
    "Akan (ak)": "ak",
    "Uighur (ug)": "ug"
}
LANGUAGES = {
    "Automatic detection": "Automatic detection",
    "Arabic (ar)": "ar",
    "Chinese - Simplified (zh-CN)": "zh",
    "Czech (cs)": "cs",
    "Danish (da)": "da",
    "Dutch (nl)": "nl",
    "English (en)": "en",
    "Finnish (fi)": "fi",
    "French (fr)": "fr",
    "German (de)": "de",
    "Greek (el)": "el",
    "Hebrew (he)": "he",
    "Hungarian (hu)": "hu",
    "Italian (it)": "it",
    "Japanese (ja)": "ja",
    "Korean (ko)": "ko",
    "Persian (fa)": "fa",  # no aux gTTS
    "Polish (pl)": "pl",
    "Portuguese (pt)": "pt",
    "Russian (ru)": "ru",
    "Spanish (es)": "es",
    "Turkish (tr)": "tr",
    "Ukrainian (uk)": "uk",
    "Urdu (ur)": "ur",
    "Vietnamese (vi)": "vi",
    "Hindi (hi)": "hi",
    "Indonesian (id)": "id",
    "Bengali (bn)": "bn",
    "Telugu (te)": "te",
    "Marathi (mr)": "mr",
    "Tamil (ta)": "ta",
    "Javanese (jw|jv)": "jw",
    "Catalan (ca)": "ca",
    "Nepali (ne)": "ne",
    "Thai (th)": "th",
    "Swedish (sv)": "sv",
    "Amharic (am)": "am",
    "Welsh (cy)": "cy",  # no aux gTTS
    "Estonian (et)": "et",
    "Croatian (hr)": "hr",
    "Icelandic (is)": "is",
    "Georgian (ka)": "ka",  # no aux gTTS
    "Khmer (km)": "km",
    "Slovak (sk)": "sk",
    "Albanian (sq)": "sq",
    "Serbian (sr)": "sr",
    "Azerbaijani (az)": "az",  # no aux gTTS
    "Bulgarian (bg)": "bg",
    "Galician (gl)": "gl",  # no aux gTTS
    "Gujarati (gu)": "gu",
    "Kazakh (kk)": "kk",  # no aux gTTS
    "Kannada (kn)": "kn",
    "Lithuanian (lt)": "lt",  # no aux gTTS
    "Latvian (lv)": "lv",
    "Macedonian (mk)": "mk",  # no aux gTTS # error get align model
    "Malayalam (ml)": "ml",
    "Malay (ms)": "ms",  # error get align model
    "Romanian (ro)": "ro",
    "Sinhala (si)": "si",
    "Sundanese (su)": "su",
    "Swahili (sw)": "sw",  # error aling
    "Afrikaans (af)": "af",
    "Bosnian (bs)": "bs",
    "Latin (la)": "la",
    "Myanmar Burmese (my)": "my",
    "Norwegian (no|nb)": "no",
    "Chinese - Traditional (zh-TW)": "zh-TW",
    "Assamese (as)": "as",
    "Basque (eu)": "eu",
    "Hausa (ha)": "ha",
    "Haitian Creole (ht)": "ht",
    "Armenian (hy)": "hy",
    "Lao (lo)": "lo",
    "Malagasy (mg)": "mg",
    "Mongolian (mn)": "mn",
    "Maltese (mt)": "mt",
    "Punjabi (pa)": "pa",
    "Pashto (ps)": "ps",
    "Slovenian (sl)": "sl",
    "Shona (sn)": "sn",
    "Somali (so)": "so",
    "Tajik (tg)": "tg",
    "Turkmen (tk)": "tk",
    "Tatar (tt)": "tt",
    "Uzbek (uz)": "uz",
    "Yoruba (yo)": "yo",
    **LANGUAGES_UNIDIRECTIONAL
}
BASE_L_LIST = LANGUAGES.keys()
LANGUAGES_LIST = [list(BASE_L_LIST)[0]] + sorted(list(BASE_L_LIST)[1:])
EXTRA_ALIGN = {
    "id": "indonesian-nlp/wav2vec2-large-xlsr-indonesian",
    "bn": "arijitx/wav2vec2-large-xlsr-bengali",
    "mr": "sumedh/wav2vec2-large-xlsr-marathi",
    "ta": "Amrrs/wav2vec2-large-xlsr-53-tamil",
    "jw": "cahya/wav2vec2-large-xlsr-javanese",
    "ne": "shniranjan/wav2vec2-large-xlsr-300m-nepali",
    "th": "sakares/wav2vec2-large-xlsr-thai-demo",
    "sv": "KBLab/wav2vec2-large-voxrex-swedish",
    "am": "agkphysics/wav2vec2-large-xlsr-53-amharic",
    "cy": "Srulikbdd/Wav2Vec2-large-xlsr-welsh",
    "et": "anton-l/wav2vec2-large-xlsr-53-estonian",
    "hr": "classla/wav2vec2-xls-r-parlaspeech-hr",
    "is": "carlosdanielhernandezmena/wav2vec2-large-xlsr-53-icelandic-ep10-1000h",
    "ka": "MehdiHosseiniMoghadam/wav2vec2-large-xlsr-53-Georgian",
    "km": "vitouphy/wav2vec2-xls-r-300m-khmer",
    "sk": "infinitejoy/wav2vec2-large-xls-r-300m-slovak",
    "sq": "Alimzhan/wav2vec2-large-xls-r-300m-albanian-colab",
    "sr": "dnikolic/wav2vec2-xlsr-530-serbian-colab",
    "az": "nijatzeynalov/wav2vec2-large-mms-1b-azerbaijani-common_voice15.0",
    "bg": "infinitejoy/wav2vec2-large-xls-r-300m-bulgarian",
    "gl": "ifrz/wav2vec2-large-xlsr-galician",
    "gu": "Harveenchadha/vakyansh-wav2vec2-gujarati-gnm-100",
    "kk": "aismlv/wav2vec2-large-xlsr-kazakh",
    "kn": "Harveenchadha/vakyansh-wav2vec2-kannada-knm-560",
    "lt": "DeividasM/wav2vec2-large-xlsr-53-lithuanian",
    "lv": "anton-l/wav2vec2-large-xlsr-53-latvian",
    "mk": "",  # Konstantin-Bogdanoski/wav2vec2-macedonian-base
    "ml": "gvs/wav2vec2-large-xlsr-malayalam",
    "ms": "",  # Duy/wav2vec2_malay
    "ro": "anton-l/wav2vec2-large-xlsr-53-romanian",
    "si": "IAmNotAnanth/wav2vec2-large-xls-r-300m-sinhala",
    "su": "cahya/wav2vec2-large-xlsr-sundanese",
    "sw": "",  # Lians/fine-tune-wav2vec2-large-swahili
    "af": "",  # ylacombe/wav2vec2-common_voice-af-demo
    "bs": "",
    "la": "",
    "my": "",
    "no": "NbAiLab/wav2vec2-xlsr-300m-norwegian",
    "zh-TW": "jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn",
    "as": "",
    "eu": "", # cahya/wav2vec2-large-xlsr-basque # verify
    "ha": "infinitejoy/wav2vec2-large-xls-r-300m-hausa",
    "ht": "",
    "hy": "infinitejoy/wav2vec2-large-xls-r-300m-armenian", # no (.)
    "lo": "",
    "mg": "",
    "mn": "tugstugi/wav2vec2-large-xlsr-53-mongolian",
    "mt": "carlosdanielhernandezmena/wav2vec2-large-xlsr-53-maltese-64h",
    "pa": "kingabzpro/wav2vec2-large-xlsr-53-punjabi",
    "ps": "aamirhs/wav2vec2-large-xls-r-300m-pashto-colab",
    "sl": "anton-l/wav2vec2-large-xlsr-53-slovenian",
    "sn": "",
    "so": "",
    "tg": "",
    "tk": "",  # Ragav/wav2vec2-tk
    "tt": "anton-l/wav2vec2-large-xlsr-53-tatar",
    "uz": "",  # Mekhriddin/wav2vec2-large-xls-r-300m-uzbek-colab
    "yo": "ogbi/wav2vec2-large-mms-1b-yoruba-test",
}
from IPython.display import clear_output
clear_output()

In [4]:
#@title utils
#@title youtube video downloader
import yt_dlp
import ffmpeg
import sys
import uuid
import re
import os
def extract_video_id(url):
    # Regular expression to extract the video ID from different YouTube URL formats
    pattern = r"(?:youtu\.be/|youtube(?:-nocookie)?\.com/(?:embed/|v/|shorts/|watch\?v=|watch\?.+&v=))([\w-]+)"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None

def download_audio(Youtube_Video_Link):
  global base_path
  yt_path=f"{base_path}/youtube_audio"
  if not os.path.exists(yt_path):
    os.makedirs(yt_path)
  os.chdir(yt_path)
  video_id = extract_video_id(Youtube_Video_Link)
  yt_url = f"https://www.youtube.com/watch?v={video_id}"
  random_uuid = str(uuid.uuid4())[:8]
  ydl_opts = {
    'format': 'bestaudio/best',
  #    'outtmpl': 'output.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
    }],
    "outtmpl": f'{random_uuid}',  # this is where you can edit how you'd like the filenames to be formatted
  }
  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([yt_url])
  os.chdir(base_path)
  return f"{yt_path}/{random_uuid}.mp3"

from audio_separator.separator import Separator
import random

def extract_vocal(audio_file):
  global base_path
  store_path=f"{base_path}/vocal"
  if not os.path.exists(store_path):
    os.makedirs(store_path)
  os.chdir(store_path)
  # Initialize the Separator with other configuration properties, below
  separator = Separator()
  # Load a model
  separator.load_model(model_filename='UVR-MDX-NET-Inst_HQ_3.onnx')
  # separator.load_model(model_filename='UVR_MDXNET_KARA_2.onnx')
  # Separate multiple audio files without reloading the model
  output_file_paths= separator.separate(audio_file)
  del separator
  gc.collect()
  torch.cuda.empty_cache()
  os.chdir(base_path)
  return f"{store_path}/{output_file_paths[0]}"


def transcribe_speech(audio_path,language_name=None):
  asr_options = {
        "initial_prompt": None,
        "suppress_numerals": True
    }

  whisperx_model = whisperx.load_model(
        "large-v3",
        "cuda" ,
        compute_type="auto",
        language=language_name,
        asr_options=asr_options,
    )
  audio = whisperx.load_audio(audio_path)
  result = whisperx_model.transcribe(
      audio,
      batch_size=16,
      chunk_size=15,
      print_progress=True,
  )
  del whisperx_model
  gc.collect()
  torch.cuda.empty_cache()  # noqa
  return audio, result

def remove_music_symbol(result):
    clean_result = {"segments": [], "language": result.get('language', '')}

    for segment in result['segments']:
        temp = {}
        for key, value in segment.items():
            if key == "text":
                # Remove the '¶' symbol
                value = value.replace("¶", "")
            temp[key] = value
        # Only append if the 'text' field is not empty after cleaning
        if temp.get("text", "").strip():
            clean_result['segments'].append(temp)

    return clean_result

def align_speech(audio, result):
    global LANGUAGES,EXTRA_ALIGN,INVERTED_LANGUAGES
    my_lang= result["language"]
    DAMHF.update(DAMT)  # lang align
    if (
        not result["language"] in DAMHF.keys()
        and not result["language"] in EXTRA_ALIGN.keys()
    ):
        print(
            "Automatic detection: Source language not compatible with align"
        )
        # raise ValueError(
        #     f"Detected language {result['language']}  incompatible, "
        #     "you can select the source language to avoid this error."
        # )
    if (
        result["language"] in EXTRA_ALIGN.keys()
        and EXTRA_ALIGN[result["language"]] == ""
    ):
        lang_name = (
            INVERTED_LANGUAGES[result["language"]]
            if result["language"] in INVERTED_LANGUAGES.keys()
            else result["language"]
        )
        # logger.warning(
        #     "No compatible wav2vec2 model found "
        #     f"for the language '{lang_name}', skipping alignment."
        # )
        return result

    model_a, metadata = whisperx.load_align_model(
        language_code=result["language"],
        device="cuda",
        model_name=None
        if result["language"] in DAMHF.keys()
        else EXTRA_ALIGN[result["language"]],
    )
    result = whisperx.align(
        result["segments"],
        model_a,
        metadata,
        audio,
        "cuda",
        return_char_alignments=True,
        print_progress=False,
    )
    del model_a
    gc.collect()
    torch.cuda.empty_cache()  # noqa
    return result,my_lang

def seperate_speaker(audio,result,min_speakers=1,max_speakers=2):
    global huggingface_token
    device = "cuda"
    batch_size = 16 # reduce if low on GPU mem
    compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)
    diarize_model = whisperx.DiarizationPipeline(use_auth_token=huggingface_token, device=device)
    # add min/max number of speakers if known
    # diarize_segments = diarize_model(audio)
    diarize_segments=diarize_model(audio, min_speakers=min_speakers, max_speakers=max_speakers)
    new_result = whisperx.assign_word_speakers(diarize_segments, result)
    del diarize_model
    gc.collect()
    torch.cuda.empty_cache()
    return diarize_segments,new_result

def get_speaker_data(diarize_segments,result):
  speaker_result={}
  unique_speakers = diarize_segments['speaker'].unique()
  for i in unique_speakers:
    speaker_result[i]=[]
  for i in result['segments']:
    try:
        speaker=i['speaker']
        temp={"text":i['text'],
        "start":i['start'],
        "end":i['end']
        }
        speaker_result[speaker].append(temp)
    except:
      pass
  new_speaker_result = {i: value for i, (key, value) in enumerate(speaker_result.items())}
  return list(new_speaker_result.keys()),new_speaker_result




###### audio Triming part

from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub import AudioSegment
from IPython.display import Audio
from IPython.core.display import display
import shutil
import os

def remove_silence(file_path, silence_threshold=0.1):
    global base_path
    store_path=f"{base_path}/vocal/remove_silence"
    if not os.path.exists(store_path):
      os.makedirs(store_path)
    f_name=os.path.basename(file_path)
    output_path=f"{store_path}/{f_name}"
    """
    Remove or keep silence from an audio file based on the given parameter.

    :param file_path: Path to the input audio file.
    :param output_path: Path to the output audio file.
    :param silence_threshold: Parameter between 0.0 and 1.0 indicating the maximum silence gap to keep.
                              0.0 means remove all silence, 1.0 means keep silence gaps of up to 1.0 seconds.
    """
    # Extract file name and format from the provided path
    file_name = os.path.basename(file_path)
    audio_format = "wav"

    # Reading the audio file
    sound = AudioSegment.from_file(file_path, format=audio_format)

    # Convert silence_threshold to milliseconds
    max_silence_len = silence_threshold * 1000  # Convert to milliseconds

    # Split the audio on silence
    audio_chunks = split_on_silence(sound,
                                    min_silence_len=1,  # Smallest chunk of silence considered
                                    silence_thresh=-45,
                                    keep_silence=max_silence_len)

    # Combine chunks into one audio segment
    combined = AudioSegment.empty()
    for chunk in audio_chunks:
        combined += chunk

    # Export the processed audio
    combined.export(output_path, format=audio_format)
    # print(f"Silence Remove")
    return output_path




def trim_audio(input_wav, start_time, end_time, output_wav,no_silence=False,silence_threshold=0.1):
    # Load the audio file
    audio = AudioSegment.from_wav(input_wav)
    # Convert seconds to milliseconds
    start_time_ms = start_time * 1000
    end_time_ms = end_time * 1000
    # Trim the audio
    trimmed_audio = audio[start_time_ms:end_time_ms]
    # Export the trimmed audio
    trimmed_audio.export(output_wav, format="wav")
    if no_silence:
      temp_path=remove_silence(output_wav,silence_threshold)
      shutil.copy(temp_path,output_wav)
    return output_wav

def check_speaker(speaker_result,select_speaker,no_silence=True,silence_threshold=0.1):
  global base_path
  random_index = random.randint(0, len(speaker_result[select_speaker])-1)
  text = speaker_result[select_speaker][random_index]['text']
  start_time = speaker_result[select_speaker][random_index]['start']
  end_time = speaker_result[select_speaker][random_index]['end']
  store_path=f"{base_path}/vocal/speaker"
  if not os.path.exists(store_path):
    os.makedirs(store_path)
  output_wav = f"{store_path}/{select_speaker}.wav"
  trim_audio(audio_path, start_time, end_time, output_wav,no_silence=no_silence,silence_threshold=silence_threshold)
  print(f"Speaker ID: {select_speaker}\nText:{text}")
  display(Audio(output_wav, autoplay=False))

def ensure_ending_punctuation(sentence):
    # List of punctuation marks used at the end of sentences
    end_punctuation = {'.', '!', '?'}
    # Strip trailing whitespace
    sentence = sentence.strip()
    # Check if the last character is a valid ending punctuation
    if sentence and sentence[-1] not in end_punctuation:
        # Add a period if there's no valid ending punctuation
        sentence += '.'
    return sentence


import os
def get_last_index(dataset_path):
  # List of files
  old_files = os.listdir(dataset_path)
  if old_files:
    # Filter to include only .wav files and remove extensions
    wav_files = [f for f in old_files if f.endswith('.wav')]
    # Sort based on file names without extension
    sorted_wav_files = sorted(wav_files, key=lambda x: os.path.splitext(x)[0])

    # Get the last file in the sorted list
    last_file = sorted_wav_files[-1] if sorted_wav_files else None
    last_index=int(last_file.replace(".wav",""))
    return last_index
  else:
    return 0

import os
import shutil

def create_melo_tts_dataset(speaker_result, audio_path,
                            voice_name, language_code_name,
                            multiple_youtube_videos=True, speaker_id=0,
                            no_silence=False, silence_threshold=0.1):

    global base_path
    dataset_path = f"{base_path}/{voice_name.strip().upper()}"
    model_name = f"{language_code_name.lower()}-{voice_name.lower()}"

    # Handling the dataset path creation or deletion based on `multiple_youtube_videos`
    if multiple_youtube_videos:
        if not os.path.exists(dataset_path):
            os.makedirs(dataset_path)
    else:
        if os.path.exists(dataset_path):
            shutil.rmtree(dataset_path)
        os.makedirs(dataset_path)

    # Open the metadata file for writing
    metadata_text_file = f"{base_path}/metadata.txt"
    if multiple_youtube_videos:
      mode='a'
    else:
      mode='w'
    with open(metadata_text_file, mode) as colab_file:
        i = get_last_index(dataset_path)
        for k, j in enumerate(speaker_result[speaker_id]):
            text = j['text'].strip()
            text = ensure_ending_punctuation(text)
            start_time = j['start']
            end_time = j['end']
            no_of_words = len(text.split())

            if no_of_words >= 3:
                f_name = f"{i:06d}"
                text_filename = f"{dataset_path}/{f_name}.txt"
                with open(text_filename, 'w') as f:
                    f.write(text)

                output_wav = f"{dataset_path}/{f_name}.wav"
                # Trim audio and save it as WAV
                trim_audio(audio_path, start_time, end_time, output_wav,
                           no_silence=no_silence, silence_threshold=silence_threshold)

                # Write the metadata
                line = f"{f_name}.wav|{model_name}|{language_code_name}|{text}\n"
                colab_file.write(line)

                i += 1
    return dataset_path,metadata_text_file,model_name



import os
import shutil
import json
from zipfile import ZipFile
import uuid

def make_zip(dataset_folder,download_zip):
    if os.path.exists(download_zip):
        os.remove(download_zip)
    with ZipFile(download_zip, 'w') as zipf:
        for root, dirs, files in os.walk(dataset_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, dataset_folder)
                zipf.write(file_path, arcname=arcname)

def create_and_zip_tts_dataset(model_name, Language_Code_Name, dataset_path, metadata_text_file):
    # Create dataset folder
    global base_path
    dataset_folder = f"{base_path}/Dataset"
    if not os.path.exists(dataset_folder):
      # shutil.rmtree(dataset_folder)
      os.mkdir(dataset_folder)
    temp_folder=f"{dataset_folder}/{os.path.basename(dataset_path)}"
    if os.path.exists(temp_folder):
      shutil.rmtree(temp_folder)
    os.makedirs(temp_folder)
    for i in os.listdir(dataset_path):
      if i.endswith(".wav"):
        shutil.copy(f"{dataset_path}/{i}",f"{temp_folder}/{i}")
    # Write config.json
    # data = {
    #     "modelname": model_name,
    #     "language": Language_Code_Name,
    #     "metadata_file": "metadata.list"
    # }
    # output_file_path = f'{temp_folder}/config.json'
    # with open(output_file_path, 'w') as file:
    #     json.dump(data, file, indent=4)
    # print(f"Data written to {output_file_path}")

    # Copy dataset files
    temp_folder_ = f"{dataset_folder}/{os.path.basename(dataset_path)}_with_text"
    if os.path.exists(temp_folder_):
        shutil.rmtree(temp_folder_)
    shutil.copytree(dataset_path, temp_folder_)
    shutil.copy(metadata_text_file, f"{temp_folder}/metadata.list")

    # Zip the folder
    random_uuid = str(uuid.uuid4())[:6]
    download_zip_with_text = f"{base_path}/{os.path.basename(dataset_path)}_with_text.zip"
    make_zip(temp_folder_,download_zip_with_text)
    download_zip = f"{base_path}/{os.path.basename(dataset_path)}.zip"
    make_zip(temp_folder,download_zip)
    print(f"Dataset zipped at {download_zip}")
    return download_zip,download_zip_with_text



In [5]:
#set up root path
import os
base_path="/content"
# base_path=os.getcwd()
base_path

'/content'

In [6]:
#@title Download a youtube video to make a audio dataset
youtbe_video_link = ""  # @param {type: "string"}
save_audio_path=download_audio(youtbe_video_link)
from IPython.display import clear_output
clear_output()
save_audio_path

'/content/youtube_audio/c66d55ca.mp3'

In [ ]:
# save_audio_path="/content/upload_audio.wav"

In [7]:
Source_Language = "English" # @param ['None','English','Hindi','Bengali','Afrikaans', 'Amharic', 'Arabic', 'Azerbaijani', 'Bulgarian', 'Bengali', 'Bosnian', 'Catalan', 'Czech', 'Welsh', 'Danish', 'German', 'Greek', 'English', 'Spanish', 'French', 'Irish', 'Galician', 'Gujarati', 'Hebrew', 'Hindi', 'Croatian', 'Hungarian', 'Indonesian', 'Icelandic', 'Italian', 'Japanese', 'Javanese', 'Georgian', 'Kazakh', 'Khmer', 'Kannada', 'Korean', 'Lao', 'Lithuanian', 'Latvian', 'Macedonian', 'Malayalam', 'Mongolian', 'Marathi', 'Malay', 'Maltese', 'Burmese', 'Norwegian Bokmål', 'Nepali', 'Dutch', 'Polish', 'Pashto', 'Portuguese', 'Romanian', 'Russian', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Albanian', 'Serbian', 'Sundanese', 'Swedish', 'Swahili', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Chinese', 'Zulu']
Min_Speakers = 1  # @param {type: "number"}
Max_Speakers = 2  # @param {type: "number"}
Separate_Background_Music = True  # @param {type: "boolean"}
if Separate_Background_Music:
  audio_path=extract_vocal(save_audio_path)
else:
  audio_path=save_audio_path
audio, result=transcribe_speech(audio_path,language_name=languages[Source_Language])
result1=remove_music_symbol(result)
result2,detect_lang=align_speech(audio, result1)
diarize_segments,result3=seperate_speaker(audio,result2,Min_Speakers,Max_Speakers)
unique_speakers,speaker_result= get_speaker_data(diarize_segments,result3)
clear_output()
print(f"Note the Speaker ID")
print(f"Numbers of Speaker is {len(unique_speakers)}")
for i in unique_speakers:
  print(f"Speaker Id: {i}")

Note the Speaker ID
Numbers of Speaker is 2
Speaker Id: 0
Speaker Id: 1


In [8]:
#@title Find Your Speaker ID
for i in unique_speakers:
  select_speaker=i
  check_speaker(speaker_result,select_speaker,no_silence=False,silence_threshold=0.1)
  print("\n")

Speaker ID: 0
Text:You're always strong, center back, strong with the good muscles in a pitch.




Speaker ID: 1
Text:We spoke on the phone when you was going to go on sign at Man United, when you was negotiating to come back.


In [9]:
#@title MelloTTS Dataset Maker
Voice_Name = "Ronaldo"  # @param {type: "string"}
Language_Code_Name = "EN"  # @param ['EN', 'ES', 'FR', 'ZH','JA','KO']
Multiple_YouTube_Videos = False  # @param {type: "boolean"}
Speaker_ID = 0  # @param {type: "number"}
No_Silence = False  # @param {type: "boolean"}
Silence_Threshold = 0.1  # @param {type: "number"}

dataset_path,metadata_text_file,model_name=create_melo_tts_dataset(speaker_result, audio_path,
                            Voice_Name, Language_Code_Name,
                            multiple_youtube_videos=Multiple_YouTube_Videos, speaker_id=Speaker_ID,
                            no_silence=No_Silence, silence_threshold=Silence_Threshold)
download_zip_path,download_zip_with_text=create_and_zip_tts_dataset(model_name, Language_Code_Name, dataset_path, metadata_text_file)


Data written to /content/Dataset/RONALDO/config.json
Dataset zipped at /content/RONALDO.zip


In [ ]:
from google.colab import files
files.download(download_zip_path)
files.download(download_zip_with_text)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
import os
import shutil

# Set the flag to save in Google Drive
Save_In_Google_Drive = True  # @param {type: "boolean"}

# Mount Google Drive if the flag is set to True
if Save_In_Google_Drive:
    drive.mount('/content/gdrive')

# Check if Google Drive is mounted
if os.path.exists("/content/gdrive/"):
    # Check if the directory already exists
    MeloTTS_dir = "/content/gdrive/MyDrive/MeloTTS"
    if not os.path.exists(MeloTTS_dir):
        os.mkdir(MeloTTS_dir)  # Create the directory if it doesn't exist

    # Define the path to save both files in the MeloTTS directory
    drive_path_zip = f"{MeloTTS_dir}/{os.path.basename(download_zip_path)}"
    drive_path_with_text = f"{MeloTTS_dir}/{os.path.basename(download_zip_with_text)}"

    # Copy the zip file to the Google Drive folder
    shutil.copy(download_zip_path, drive_path_zip)
    shutil.copy(download_zip_with_text, drive_path_with_text)

    print(f"Files saved at:\n{drive_path_zip}\n{drive_path_with_text}")
else:
    print("Google Drive not mounted.")
